#### _Programming Exercise 2.4.1:_ Compute a taptweak from a taptree

In [ ]:
TAPSCRIPT_VER = bytes([0xc0]) # See TapScript chapter for more details.
internal_pubkey = ECPubKey()
internal_pubkey.set(bytes.fromhex('03af455f4989d122e9185f8c351dbaecd13adca3eef8a9d38ef8ffed6867e342e3'))

# Derive pay-to-pubkey scripts
privkeyA, pubkeyA = generate_key_pair()
privkeyB, pubkeyB = generate_key_pair()
privkeyC, pubkeyC = generate_key_pair()
scriptA = CScript([pubkeyA.get_bytes(), OP_CHECKSIG])
scriptB = CScript([pubkeyB.get_bytes(), OP_CHECKSIG])
scriptC = CScript([pubkeyC.get_bytes(), OP_CHECKSIG])

# Method: Returns Tagged Hash
def tagged_hash(tag, input_data):
    data = sha256(tag.encode('utf-8'))
    data += data
    data += input_data
    return sha256(data)

# Method: Returns TapBranch hash
def tapbranch(taggedhash_left, taggedhash_right):
    if taggedhash_left > taggedhash_right:
        taggedhash_left, taggedhash_right = taggedhash_right, taggedhash_left
    return tagged_hash("TapBranch", taggedhash_left + taggedhash_right)

# 1) Compute TapLeafs A, B and C
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)
hash_inputC = TAPSCRIPT_VER + ser_string(scriptC)
taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)
taggedhash_leafC = tagged_hash("TapLeaf", hash_inputC)

# 2) Compute Internal node TapBranch AB
internal_nodeAB = tapbranch(taggedhash_leafA, taggedhash_leafB)

# 3) Compute TapTweak
rootABC = tapbranch(internal_nodeAB, taggedhash_leafC)
taptweak = tagged_hash("TapTweak", internal_pubkey.get_bytes() + rootABC)
print("TapTweak:", taptweak.hex())

# 4) Derive the segwit output address
taproot_pubkey_b = internal_pubkey.tweak_add(taptweak).get_bytes()
taproot_pubkey_v1 = bytes([taproot_pubkey_b[0] & 1]) + taproot_pubkey_b[1:]
segwit_address = program_to_witness(1, taproot_pubkey_v1)
print('Segwit address:', segwit_address)

#### _Programming Exercise 2.4.5_ - Constructing a taproot output from a taptree

In [ ]:
# Generate key pairs for internal pubkey and pay-to-pubkey tapscripts
privkey_internal, pubkey_internal = generate_key_pair()

privkeyA, pubkeyA = generate_key_pair()
privkeyB, pubkeyB = generate_key_pair()
privkeyC, pubkeyC = generate_key_pair()
privkeyD, pubkeyD = generate_key_pair()

# Construct Pay-to-Pubkey TapLeafs and Taptree.
TapLeafA = TapLeaf()
TapLeafB = TapLeaf()
TapLeafC = TapLeaf()
TapLeafD = TapLeaf()
TapLeafA.construct_pk(pubkeyA)
TapLeafB.construct_pk(pubkeyB)
TapLeafC.construct_pk(pubkeyC)
TapLeafD.construct_pk(pubkeyD)

# Create a Taptree with tapleafs and huffman constructor.
# Method: TapTree.huffman_constructor(tuple_list)
taptree = TapTree()
taptree.key = pubkey_internal
taptree.huffman_constructor([(1, TapLeafA), (1, TapLeafB), (1, TapLeafC), (1, TapLeafD)])

# Generate taproot tree with the `construct()` method, then use the taproot bytes to create a segwit address
taproot_script, tweak, control_map = taptree.construct()
program = bytes(taproot_script[2:])
address = program_to_witness(1, program)
print("Address: {}".format(address))

#### _Programming Exercise 2.4.8:_ Sign the transaction for `TapLeafA` 

In [ ]:
# Generate the Taproot Signature Hash for signing
sighashA = TaprootSignatureHash(spending_tx,
                               [tx.vout[0]],
                               SIGHASH_ALL_TAPROOT,
                               input_index=0,
                               scriptpath=True,
                               tapscript=TapLeafA.script)

signatureA = privkeyA.sign_schnorr(sighashA)

print("Signature for TapLeafA: {}\n".format(signatureA.hex()))

#### _Programming Exercise  2.4.9:_ Construct the witness, add it to the transaction and verify mempool acceptance

In [ ]:
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness = CScriptWitness()
witness.stack = [signatureA, TapLeafA.script, control_map[TapLeafA.script]]
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")